# AdaRank implementation

In this notebook we show a simple and effective implementation of AdaRank algorithm from paper [AdaRank: a boosting algorithm for information retrieval](https://dl.acm.org/doi/abs/10.1145/1277741.1277809?casa_token=ku7AGgHMiTsAAAAA:sm_rUCTguz9F5k2yANn2iLGLVwBFpMkQOB_zN9csd7zCH5mDSYBFOToYoQmP5ChVbOUidtZWFFgZxw) for listwise ranking of documents with respect to queries.

## 1. Initialization

In this first part we setup the libraries installation.

In [ ]:
%pip install pandas numpy sklearn

In [177]:
import numpy as np
import os
import pandas as pd
import random
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.base import BaseEstimator
from typing import Union

Here we convert the raw csv dataset files into the expected format for the AdaRank algorithm. We choose to create a file of index pairs for (query, document) in order to avoid the redundancy given by queries and documents text.


In [178]:
# list of document datasets for each query
documents = []
# list of queries
queries = []
# list of datasets of pair (query, document) with a random relevance score in [0, 100)
pairs = []
# progessive indices used to keep count
dindex, qindex = 0, 0
# each raw dataset is used to embody its data in a new format
for file in os.listdir('raw'):
    frame = pd.read_csv(os.path.join('raw', file), sep = ',')
    query = file.removesuffix('.csv').replace('_', ' ')
    # insert query and related documents
    queries.append(query)
    documents.append(frame.drop(columns = [ 'label' ]))
    # we insert (query-index, document-index, random relevance score) for each document retrieved in the query
    pairs.extend([ (qindex, dindex + d, row.label) for d, row in enumerate(frame.itertuples()) ])
    # progressive update of indices
    qindex += 1
# concatenation among all queries of the documents
documents = pd.concat(documents, axis = 0, ignore_index = True)
documents.to_csv('data/documents.csv', sep = ',')
# concatenation of all queries
queries = pd.DataFrame({ 'text': queries })
queries.to_csv('data/queries.csv', sep = ',', index_label = 'query')
# concatenation of all pairs with relevance scores in a new file
pairs = pd.DataFrame(pairs, columns = [ 'query', 'document', 'label' ])
pairs.to_csv('data/pairs.csv', sep = ',')

pairs

,query,document,label
0,0,0,25
1,0,1,60
2,0,2,21
3,0,3,66
4,0,4,24
...,...,...,...
196,2,62,58
197,2,63,35
198,2,64,57
199,2,65,14


## 2. Feature extraction

For our feature extraction mecahnism we combine each document and query by extracting three features.

| Feature                   | Computation                                                                             |
|---------------------------|-----------------------------------------------------------------------------------------|
| log-frecency sum          | $\sum_{\forall w_j \, \in \, d \, \cup \, q} \log(1 + \text{count}(w_j, d)) $           |
| log-frecency sum          | $\sum_{\forall w_j \, \in \, d \, \cup \, q} \log(\text{tf-idf}(w_j, d)) $              |
| log-inverse-frequency sum | $\sum_{\forall w_j \, \in \, d \, \cup \, q} \log(1 + 1 / (1 + \text{count}(w_j, d))) $ |

In [179]:
features = None
labels = None
queries_words = queries['text'].astype('string').str.lower().str.split()
# we build vocabularies and features
for query, indices in pairs.groupby('query').groups.items():
    # we select documents text for current query
    # documents_selected = documents.loc[pairs.loc[indices, 'document'], 'name']
    documents_selected = documents.loc[pairs.loc[indices, 'document'], 'name']
    # and all the tokenized words
    documents_words = TfidfVectorizer(stop_words = 'english', binary = False).fit(documents_selected).get_feature_names_out()
    # to such vocabulary we add tokens from the query's text
    vocabulary = set(documents_words.tolist()).union(queries_words.iloc[query])
    # we build the design matrix for current query with all fancy features
    matrix = pd.DataFrame({
        'query': query,
        'log-frecency': pd.DataFrame(
            CountVectorizer(stop_words = 'english', binary = False, vocabulary = vocabulary).fit_transform(documents_selected).todense()
        ).apply(lambda x: np.log(1 + x)).sum(axis = 1),
        'tf-idf': pd.DataFrame(
            TfidfVectorizer(stop_words = 'english', binary = False, use_idf = True, vocabulary = vocabulary).fit_transform(documents_selected).todense(), 
        ).sum(axis = 1),
        'log-inverse-frequency': pd.DataFrame(
            CountVectorizer(stop_words = 'english', binary = False, vocabulary = vocabulary).fit_transform(documents_selected).todense()
        ).apply(lambda x: np.log(1 + 1 / (1 + x))).sum(axis = 1),
    })
    # we concatenate each feature matrix
    if features is None:
        features = matrix
    else:
        features = pd.concat([ features, matrix ], axis = 0, ignore_index = True)
    # we concatenate each label vector
    if labels is None:
        labels = pairs.loc[indices, 'label']
    else:
        labels = pd.concat([ labels, pairs.loc[indices, 'label'] ], axis = 0, ignore_index = True)

features

,query,log-frecency,tf-idf,log-inverse-frequency
0,0,2.772589,1.947576,77.868050
1,0,4.852030,2.465639,77.005004
2,0,4.158883,2.325861,77.292686
3,0,2.772589,1.844363,77.868050
4,0,1.386294,1.377624,78.443414
...,...,...,...,...
196,2,4.852030,2.465639,78.391298
197,2,4.158883,2.226099,78.678981
198,2,4.158883,2.283433,78.678981
199,2,3.465736,2.224207,78.966663


## 3. AdaRank implementation

For our implementation we follow the explanation from the paper and we extend Scikit-Learn base class. Besides, we use NDCG score for evaluating the performance of weak rankers.

In [180]:
class AdaRank(BaseEstimator):

    def __init__(self, n_rounds: int = 10, metric: str = 'ndcg') -> None:
        super().__init__()
        # number of training rounds
        self.n_rounds = n_rounds
        # performance metrics, by default NDCG
        self.scorer_ = ndcg_score if metric else None
        # alphas coefficients for weighting rankers' importances
        self.alphas_ = np.empty(n_rounds, dtype = np.float32)
        # rankers here are the indices of the column selected
        # at each iteration to maximize the performance score
        self.rankers_ = np.empty(n_rounds, dtype = np.int32)

    def fit(self, X: pd.DataFrame, y: Union[pd.DataFrame, pd.Series]):
        def rank_by_feature(X: pd.DataFrame):
            return X.reset_index(drop = True).sort_values().reset_index().sort_values('index').index
        # number of queries in the dataset
        self.n_queries_ = len(X.groupby('query').groups)
        # features (minus query column)
        n_features = X.shape[1] - 1
        # distribution coefficients for weighting queries' importances (sum up to 1)
        distributions = np.ones(self.n_queries_, dtype = np.float32) / self.n_queries_
        # (query-index, document-indices) dictionary
        query_groups = X.groupby('query').groups
        # true relevance labels of each document for each query
        targets = [ y.iloc[indices].tolist() for indices in query_groups.values() ]
        # final boosting (additive) criterion for each query (used for ranking)
        criterion = [ np.zeros(len(indices)) for indices in query_groups.values() ]
        # iterations
        for t in range(self.n_rounds):
            # search feature that maximizes the score
            max_score = -np.inf
            argmax_feature = None
            argmax_scores = None
            # compute score for each one
            for k in range(1, 1 + n_features):
                # document indices permutations (for each query)
                permutations = [ rank_by_feature(X.iloc[indices, k]) for indices in query_groups.values() ]
                # compute performance scores for each feature (for each query) in [-1, 1]
                scores = 2 * np.array([ 
                        self.scorer_(
                            [ targets[iquery] ], 
                            [ permutations[iquery] ]
                        ) 
                        for iquery in range(self.n_queries_) 
                    ], 
                    dtype = np.float32
                ) - 1
                # aggregate scores over the query
                score = np.dot(scores, distributions)
                # update if better
                if score > max_score:
                    max_score = score
                    argmax_feature = k
                    argmax_scores = scores
            # ranker column index at current iteration
            self.rankers_[t] = argmax_feature
            # compute alpha for current iteration
            self.alphas_[t] = 0.5 * np.log(np.dot(distributions, 1 + argmax_scores) / np.dot(distributions, 1 - argmax_scores))
            # update boosting criterion
            for iquery in range(self.n_queries_):
                criterion[iquery] += self.alphas_[t] * X.iloc[query_groups[iquery], self.rankers_[t]]
            # recompute performance scores with boosting criterion
            scores = 2 * np.array([ 
                    self.scorer_(
                        [ targets[iquery] ],
                        [ rank_by_feature(criterion[iquery]) ]
                    )
                    for iquery in range(self.n_queries_) 
                ],
                dtype = np.float32
            ) - 1
            # update distribution weights
            distributions = np.exp(-scores) / np.sum(np.exp(-scores))

    def transform(self, X, y = None):
        outputs = []
        query_groups = X.groupby('query').groups
        criterion = [ np.zeros(len(indices)) for indices in query_groups.values() ]
        n_ranked = 0
        # compute boosting criterion with alphas and rankers (best columns) for each iteration
        for alpha, ranker in zip(self.alphas_, self.rankers_):
            for iquery, indices in query_groups.items():
                criterion[iquery] += alpha * X.iloc[indices, ranker]
        # within each query sort the documents
        for iquery, indices in query_groups.items():
            ranking = pd.DataFrame({ 'query': iquery, 'document': n_ranked + np.argsort(criterion[iquery]) })
            n_ranked += len(ranking)
            outputs.append(ranking)
        # concatenate all sorted indices (by relevance) in one unique frame
        return pd.concat(outputs, axis = 0)

## 4. Usage

Before feeding our design matrix to AdaRank, we scale features using common standardization.

In [181]:
# standardized transformed features in [-1, 1] with 0 standard deviation
features.iloc[:, 1:] = StandardScaler().fit_transform(features.iloc[:, 1:])

Here we train the model and we emit the ranking of documents (with respect to original indices)

In [182]:
model = AdaRank()
# train the model on ground truth labels
model.fit(features, labels)
# emit the ranking
ranking = model.transform(features)
ranking

,query,document
0,0,29
1,0,6
2,0,46
3,0,11
4,0,40
...,...,...
196,2,184
197,2,172
198,2,153
199,2,138


This is how documents appear sorted with respect to the query index according to our implementation of AdaRank.

In [183]:
first_query = pairs[pairs['query'] == 0]
first_query_documents = pd.concat((documents.iloc[first_query.index, :], first_query), axis = 1)
first_query_ranking = ranking[ranking['query'] == 0]
ranked = first_query_documents.iloc[first_query_ranking['document'], :]
# show results of first query, namely 'birulin in plasma'
ranked[ranked['query'] == 0]

,loinc,name,component,system,property,query,document,label
29,20442-0,Hepatitis B virus DNA [#/volume] (viral load) ...,Hepatitis B virus DNA,Ser,NCnc,0,29,28
6,13317-3,Methicillin resistant Staphylococcus aureus [P...,Staphylococcus aureus.methicillin resistant is...,XXX,ACnc,0,6,59
46,14764-5,Glucose [Moles/volume] in Serum or Plasma --3 ...,Glucose^3H post 100 g glucose PO,Ser/Plas,SCnc,0,46,47
11,30522-7,C reactive protein [Mass/volume] in Serum or P...,C reactive protein,Ser/Plas,MCnc,0,11,58
40,1003-3,Indirect antiglobulin test.complement specific...,Indirect antiglobulin test.complement specific...,Ser/Plas,ACnc,0,40,13
...,...,...,...,...,...,...,...,...
50,20629-2,Levofloxacin [Susceptibility],Levofloxacin,Isolate,Susc,0,50,29
38,18928-2,Gentamicin [Susceptibility],Gentamicin,Isolate,Susc,0,38,61
19,19000-9,Vancomycin [Susceptibility],Vancomycin,Isolate,Susc,0,19,56
4,18906-8,Ciprofloxacin [Susceptibility],Ciprofloxacin,Isolate,Susc,0,4,24
